失球数分析

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
matches = spark.read.format("csv").option("header",True).option("encoding","utf-8").option("sep",",").load("../dataset/WorldCupMatches.csv")
matches.createTempView("goalsGame")
matches.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Datetime: string (nullable = true)
 |-- Stage: string (nullable = true)
 |-- Stadium: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Home Team Name: string (nullable = true)
 |-- Home Team Goals: string (nullable = true)
 |-- Away Team Goals: string (nullable = true)
 |-- Away Team Name: string (nullable = true)
 |-- Attendance: string (nullable = true)
 |-- Half-time Home Goals: string (nullable = true)
 |-- Half-time Away Goals: string (nullable = true)
 |-- Referee: string (nullable = true)
 |-- Assistant 1: string (nullable = true)
 |-- Assistant 2: string (nullable = true)
 |-- RoundID: string (nullable = true)
 |-- MatchID: string (nullable = true)
 |-- Home Team Initials: string (nullable = true)
 |-- Away Team Initials: string (nullable = true)


In [2]:
total_goals_conceded = spark.sql('''
SELECT country,SUM(goals_conceded) AS goals_conceded FROM 
(SELECT `Home Team Name` AS country,SUM(CAST(`Away Team Goals` AS Integer)) AS goals_conceded FROM goalsGame GROUP BY country
 UNION ALL
SELECT `Away Team Name` AS country,SUM(CAST(`Home Team Goals` AS Integer)) AS goals_conceded FROM goalsGame GROUP BY country ) t
GROUP BY country ORDER BY goals_conceded DESC LIMIT 10
''')

In [3]:
total_goals_conceded.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/worldcup") \
    .option("dbtable", "total_goals_conceded") \
    .option("user", "root") \
    .option("password", "") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .mode("append") \
    .save()
total_goals_conceded.show()

+--------------+--------------+
|       country|goals_conceded|
+--------------+--------------+
|        Brazil|           114|
|        Mexico|            94|
|     Argentina|            85|
|         Italy|            77|
|    Germany FR|            77|
|       Uruguay|            73|
|        France|            72|
|        Sweden|            69|
|       Belgium|            68|
|Korea Republic|            67|
+--------------+--------------+
